# Simon's algorithm

Simon's algorithm is closely related in principle to the Bernstein-Vazirani algorithm and the Deutsch-Josza algorithm, (which are also explained in other notebooks in this repository).

We are given an "oracle" and promised that its outputs are either **one-to-one** or **two-to-one**, i.e., it either has **exactly** as many possible outputs as possible inputs or **exactly half** as many possible outputs as possible inputs. In effect, all two-to-one functions can be thought of as being based on a "**hidden string**" encoded into the oracle, whereas one-to-one functions are the only case where the hidden string is "**0**." Our problem is therefore to determine the "hidden string" encoded into the oracle.

A classical computer, on average, needs to check about half the possible inputs to answer the question of what the hidden string is; a quantum computer needs **exponentially fewer** queries of the oracle to determine the same answer.

(...So the conventional disciplinary thinking goes, at least. But notice that we are doing 120 qubits of Simon's algorithm below, and we could argue we only need a single query, with PyQrack!)

In [1]:
from collections import Counter
from pyqrack import QrackSimulator

hidden_bits = 37
input_size = 60
num_qubits = input_size * 2
oracle_qubits = [*range(input_size)]

You may change the `hidden_bits` above to be any value that can fit within the oracle register width. Your `hidden_bits` corresponds to the oracle below:

In [2]:
def oracle(sim):
    # The first portion of this oracle is effectively a one-to-one oracle,
    # in itself, (though not the only possible one-to-one oracle).
    for i in oracle_qubits:
        sim.mcx([i], i + input_size)

    # The second portion of this oracle makes it two-to-one,
    # (unless "hidden_bits" is 0):
    for i in oracle_qubits:
        if (hidden_bits >> i) & 1:
            for j in oracle_qubits:
                if (hidden_bits >> j) & 1:
                    sim.mcx([i], j + input_size)
            break;

All of the following statements of equality are true:

In [3]:
# Prepare the initial register state:
sim = QrackSimulator(num_qubits)
for i in oracle_qubits:
    sim.h(i)

# Make exactly one query to the oracle:
oracle(sim)

# Finish the unitary portion of the algorithm, with the result from the oracle...
# (It is often presented that measurement of the second register needs to occur before this step;
# this is actually wholly theoretically unnecessary in the ideal, due to locality of quantum information.)
for i in oracle_qubits:
    sim.h(i)

# The cost in "shot" count is linear, at this point.
results = Counter(sim.measure_shots(oracle_qubits, 10000))

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir


...The ultimate point of the algorithm is that, by solving the system of equations below, **where the value of `hidden_string` is an algebraic unknown**, we can determine the value of `hidden_string`, with a probabilistic likelihood that converges exponentially faster than a "classical" algorithm, in the number of measurement "shots." (In a real hardware quantum computer, the number of "shots" would correspond to exactly the number of queries of the oracle).

Let's check if our implied system of equations is right:

In [4]:
for key in results.keys():
    x = (bin(hidden_bits & key).count("1")) & 1
    print(hidden_bits, ".", key, "=", x, "(mod 2)")

37 . 3586605405 = 0 (mod 2)
37 . 3051506437 = 0 (mod 2)
37 . 14930428 = 0 (mod 2)
37 . 2416041090 = 0 (mod 2)
37 . 2790688166 = 0 (mod 2)
37 . 3633780908 = 0 (mod 2)
37 . 690186335 = 0 (mod 2)
37 . 601054936 = 0 (mod 2)
37 . 39526329 = 0 (mod 2)
37 . 3313852869 = 0 (mod 2)
37 . 3026235269 = 0 (mod 2)
37 . 2851494658 = 0 (mod 2)
37 . 1738171558 = 0 (mod 2)
37 . 4097602837 = 0 (mod 2)
37 . 3665261194 = 0 (mod 2)
37 . 293580696 = 0 (mod 2)
37 . 2190295322 = 0 (mod 2)
37 . 3631011374 = 0 (mod 2)
37 . 2930956774 = 0 (mod 2)
37 . 2988904801 = 0 (mod 2)
37 . 3636084058 = 0 (mod 2)
37 . 4081661469 = 0 (mod 2)
37 . 143170823 = 0 (mod 2)
37 . 538325668 = 0 (mod 2)
37 . 10663476 = 0 (mod 2)
37 . 2184571279 = 0 (mod 2)
37 . 3346907162 = 0 (mod 2)
37 . 2544845250 = 0 (mod 2)
37 . 1353300879 = 0 (mod 2)
37 . 2374832135 = 0 (mod 2)
37 . 1770698821 = 0 (mod 2)
37 . 767787549 = 0 (mod 2)
37 . 3534602576 = 0 (mod 2)
37 . 1095963363 = 0 (mod 2)
37 . 2013590563 = 0 (mod 2)
37 . 626829391 = 0 (mod 2)
37 . 